In [ ]:

# IMPORTANT: RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES
# TO THE CORRECT LOCATION (/kaggle/input) IN YOUR NOTEBOOK,
# THEN FEEL FREE TO DELETE THIS CELL.
# NOTE: THIS NOTEBOOK ENVIRONMENT DIFFERS FROM KAGGLE'S PYTHON
# ENVIRONMENT SO THERE MAY BE MISSING LIBRARIES USED BY YOUR
# NOTEBOOK.

import os
os.environ['KAGGLE_CONFIG_DIR'] = '/content'
import sys
from tempfile import NamedTemporaryFile
from urllib.request import urlopen
from urllib.parse import unquote, urlparse
from urllib.error import HTTPError
from zipfile import ZipFile
import tarfile
import shutil

CHUNK_SIZE = 40960
DATA_SOURCE_MAPPING = 'blood-cells:https%3A%2F%2Fstorage.googleapis.com%2Fkaggle-data-sets%2F9232%2F29380%2Fbundle%2Farchive.zip%3FX-Goog-Algorithm%3DGOOG4-RSA-SHA256%26X-Goog-Credential%3Dgcp-kaggle-com%2540kaggle-161607.iam.gserviceaccount.com%252F20240831%252Fauto%252Fstorage%252Fgoog4_request%26X-Goog-Date%3D20240831T122623Z%26X-Goog-Expires%3D259200%26X-Goog-SignedHeaders%3Dhost%26X-Goog-Signature%3D195d0457e17e161835b1723ef8acac2821d26fdfe2462085ee43276c87b34a7d10c448f6968a0528caf002a53f490cc390c514014e4172229de4678ba23f1ea0d848b4374d24bf1847fd0c8d3c94fb9f207f1fb8a4c1a43858387cde182404e5c7cb1976f508e68c19ab8b221d928e649ee25fcfeb6cf94b1add99e3105d9c61f5f5c290249690a4f4b8cb4d423271f40639eaef84618bd9868aa200ff341c3b6749de0942070c5f87fd7b515fd7d18d9fe04dbe3e0cb56e394b9cabe0794940d55e76e09b6d0478abb69913846515acde53dfee748f1322bba2c4fa654c470319a4124a108af0ee71618bcf4bf2af91fc44ad7a87fd258c2c55d59125e0f19f'

KAGGLE_INPUT_PATH='/kaggle/input'
KAGGLE_WORKING_PATH='/kaggle/working'
KAGGLE_SYMLINK='kaggle'

!umount /kaggle/input/ 2> /dev/null
shutil.rmtree('/kaggle/input', ignore_errors=True)
os.makedirs(KAGGLE_INPUT_PATH, 0o777, exist_ok=True)
os.makedirs(KAGGLE_WORKING_PATH, 0o777, exist_ok=True)

try:
  os.symlink(KAGGLE_INPUT_PATH, os.path.join("..", 'input'), target_is_directory=True)
except FileExistsError:
  pass
try:
  os.symlink(KAGGLE_WORKING_PATH, os.path.join("..", 'working'), target_is_directory=True)
except FileExistsError:
  pass

for data_source_mapping in DATA_SOURCE_MAPPING.split(','):
    directory, download_url_encoded = data_source_mapping.split(':')
    download_url = unquote(download_url_encoded)
    filename = urlparse(download_url).path
    destination_path = os.path.join(KAGGLE_INPUT_PATH, directory)
    try:
        with urlopen(download_url) as fileres, NamedTemporaryFile() as tfile:
            total_length = fileres.headers['content-length']
            print(f'Downloading {directory}, {total_length} bytes compressed')
            dl = 0
            data = fileres.read(CHUNK_SIZE)
            while len(data) > 0:
                dl += len(data)
                tfile.write(data)
                done = int(50 * dl / int(total_length))
                sys.stdout.write(f"\r[{'=' * done}{' ' * (50-done)}] {dl} bytes downloaded")
                sys.stdout.flush()
                data = fileres.read(CHUNK_SIZE)
            if filename.endswith('.zip'):
              with ZipFile(tfile) as zfile:
                zfile.extractall(destination_path)
            else:
              with tarfile.open(tfile.name) as tarfile:
                tarfile.extractall(destination_path)
            print(f'\nDownloaded and uncompressed: {directory}')
    except HTTPError as e:
        print(f'Failed to load (likely expired) {download_url} to path {destination_path}')
        continue
    except OSError as e:
        print(f'Failed to load {download_url} to path {destination_path}')
        continue

print('Data source import complete.')


In [ ]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator, load_img, img_to_array
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, Input, InputLayer, BatchNormalization,UpSampling2D
from tensorflow.keras.layers import LeakyReLU
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.optimizers import RMSprop

import matplotlib.pyplot as plt
import numpy as np
from scipy.ndimage import median_filter

In [ ]:
pip install --upgrade tensorflow


In [ ]:
import tensorflow as tf

img_size = 128
batch_size = 64

"""train_datagen = ImageDataGenerator(
    preprocessing_function=lambda x: median_filter(x,size=(2, 2), mode='constant', cval=0),
    rescale = 1/255.#, brightness_range = [0.5,1.5],rotation_range=20,shear_range=10, zoom_range = 0.2,
    #width_shift_range = 0.15, height_shift_range = 0.15, horizontal_flip = True
)"""
train_datagen = ImageDataGenerator(
    #reprocessing_function=lambda x: median_filter(x, size=(2, 2, 3), mode='constant', cval=0),
    rescale = 1./255,brightness_range = [0.5,1.5],rotation_range=0.2,shear_range=0.3, zoom_range = 0.2,
    width_shift_range = 0.2, height_shift_range = 0.2,
    horizontal_flip = True
)
"""train_datagen = ImageDataGenerator(
    preprocessing_function=lambda x: median_filter(x,size=(2, 2), mode='constant', cval=0),
    rescale = 1./255,
    horizontal_flip = True)"""

val_datagen = ImageDataGenerator(rescale = 1./255)
test_datagen = ImageDataGenerator(rescale = 1./255.)


train_generator = train_datagen.flow_from_directory('/kaggle/input/blood-cells/dataset2-master/dataset2-master/images/TRAIN',
                                                   target_size = (img_size, img_size),
                                                   batch_size = batch_size,
                                                   shuffle=True,
                                                   class_mode='categorical')

val_generator = val_datagen.flow_from_directory('/kaggle/input/blood-cells/dataset2-master/dataset2-master/images/TEST_SIMPLE',
                                                   target_size = (img_size, img_size),
                                                   batch_size = batch_size,
                                                   shuffle=False,
                                                   class_mode='categorical')

test_generator = test_datagen.flow_from_directory('/kaggle/input/blood-cells/dataset2-master/dataset2-master/images/TEST',
                                                   target_size = (img_size, img_size),
                                                   batch_size = batch_size,
                                                   shuffle=False,
                                                   class_mode = 'categorical')

In [ ]:
#augumented images used for training
import matplotlib.pyplot as plt

augmented_images, _ = next(train_generator)


plt.figure(figsize=(10, 10))
for i in range(9):
    ax = plt.subplot(3, 3, i + 1)
    plt.imshow(augmented_images[i])
    plt.axis("off")
plt.show()


In [ ]:
input_layer = Input(shape=(128, 128, 3))

x = Conv2D(6,
           kernel_size=(1,1),
           strides=(1,1),
           padding='same',
           activation='relu')(input_layer)

x = BatchNormalization()(x)
x = LeakyReLU(alpha=0.3)(x)
x = MaxPooling2D((2, 2))(x)

x_image_1 = UpSampling2D(size=(2, 2))(x)

x = Conv2D(16, kernel_size=(5,5), strides=(1,1),padding='same',activation='relu')(x)
x = BatchNormalization()(x)
x = LeakyReLU(alpha=0.3)(x)
x = MaxPooling2D((2, 2))(x)

x_image_2 = UpSampling2D(size=(4, 4))(x)

x = Conv2D(64, kernel_size=(5,5), strides=(1,1),padding='same',activation='relu')(x)
x = BatchNormalization()(x)
x = LeakyReLU(alpha=0.3)(x)
x = MaxPooling2D((2, 2))(x)

x_image_3 = UpSampling2D(size=(8, 8))(x)

x = Conv2D(128, kernel_size=(5,5), strides=(1,1),padding='same',activation='relu')(x)
x = BatchNormalization()(x)
x = LeakyReLU(alpha=0.3)(x)
x = MaxPooling2D((2, 2))(x)

x_image_4 = UpSampling2D(size=(16, 16))(x)

x = Conv2D(512, kernel_size=(4,4), strides=(1,1),padding='same',activation='relu')(x)
x = BatchNormalization()(x)
x = LeakyReLU(alpha=0.3)(x)
x = Dropout(0.2)(x)

x = Flatten()(x)
output_layer = Dense(4, activation='softmax')(x)

model = Model(inputs=input_layer,outputs=output_layer)

layer_outputs = [x_image_1, x_image_2, x_image_3, x_image_4]
image_output_model = Model(inputs=input_layer, outputs=layer_outputs)

example_input = np.random.random((1, 128, 128, 3))

# Get the outputs of each layer for the example input
outputs = image_output_model.predict(example_input)

# Plot and display the images
for i, output in enumerate(outputs):
    plt.figure(figsize=(12, 6))
    num_channels = output.shape[-1]
    for j in range(num_channels):
        plt.subplot(1, num_channels, j + 1)
        plt.imshow(output[0, :, :, j])
        plt.axis('off')
    plt.suptitle(f"Feature Maps of Layer {i+1}")
    plt.show()

In [ ]:
model.summary()

In [ ]:
train_generator.class_indices

In [ ]:
#decayed_lr = tf.train.exponential_decay(learning_rate,
                                        #global_step, 10000,
                                        #0.95, staircase=True)
#opt = tf.train.AdamOptimizer(decayed_lr, epsilon=adam_epsilon)
"""model.compile(
    loss = 'categorical_crossentropy',
    optimizer = Adam(lr = 0.001,weight_decay=0.10),
    metrics = ["accuracy"]
)"""
initial_learning_rate = 0.001
decay_steps = 1000
decay_rate = 0.9
grad_decay = 0.1
optimizer = tf.keras.optimizers.Adam(
    learning_rate=tf.keras.optimizers.schedules.ExponentialDecay(
        initial_learning_rate=initial_learning_rate,
        decay_steps=decay_steps,
        decay_rate=decay_rate
    ),
    beta_1=1.0 - grad_decay  # Set beta_1 to 0.9 for 0.1 gradient decay
)
model.compile(
    optimizer=optimizer,
    loss = 'categorical_crossentropy',
    metrics=['accuracy']
)



In [ ]:
from tensorflow.keras.models import load_model

#model = load_model("/kaggle/input/resnet-50/tensorflow2/classification/1")


history = model.fit(
    train_generator,
    validation_data = val_generator,
    epochs = 20
)

In [ ]:
history_dict = history.history
train_acc = history_dict['loss']
val_acc = history_dict['val_loss']
epochs = range(1, len(history_dict['loss'])+1)
plt.plot(epochs, train_acc,'b', label='Training error')
plt.plot(epochs, val_acc,'b', color="orange", label='Validation error')
plt.title('Training and Validation error')
plt.xlabel('Epochs')
plt.ylabel('Error')
plt.legend()
plt.show()

In [ ]:
history_dict = history.history
train_acc = history_dict['accuracy']
val_acc = history_dict['val_accuracy']
epochs = range(1, len(history_dict['accuracy'])+1)
plt.plot(epochs, train_acc,'b', label='Training accuracy')
plt.plot(epochs, val_acc,'b', color="orange", label='Validation accuracy')
plt.title('Training and Validation accuracy')
plt.xlabel('Epochs')
plt.ylabel('accuracy')
plt.legend()
plt.show()

In [ ]:
y_preds = model.predict(test_generator)

In [ ]:


"""x, y = test_generator.next()
y_true = y
for i in range(2487//64):
    x, y = next(test_generator)
    y_true = np.concatenate([y_true, y], axis = 0)

print(y_true)"""
"""x, y = test_generator.next()
y_true = y
for i in range(2487//64):
    x, y = test_generator.next()
    y_true = np.concatenate([y_true, y], axis=0)

print(y_true)"""

test_generator = test_datagen.flow_from_directory('/kaggle/input/blood-cells/dataset2-master/dataset2-master/images/TEST',
                                                   target_size=(img_size, img_size),
                                                   batch_size=batch_size,
                                                   shuffle=False,
                                                   class_mode='categorical')

y_true = []
for i in range(2487 // batch_size):
    x, y = next(test_generator)
    y_true.append(y)

y_true = np.concatenate(y_true, axis=0)
print(y_true)

In [ ]:
y_true = np.argmax(y_true, axis = 1)
y_preds = np.argmax(y_preds, axis = 1)

print(y_true, y_preds)

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix


y_preds = y_preds[:len(y_true)]
cm = confusion_matrix(y_true, y_preds)
ax= plt.subplot()
sns.heatmap(cm, annot=True, fmt='g', ax=ax)

ax.set_xlabel('Predicted labels')
ax.set_ylabel('True labels')
ax.set_title('Confusion Matrix')
ax.xaxis.set_ticklabels(['EOSINOPHIL', 'LYMPHOCYTE', 'MONOCYTE', 'NEUTROPHIL'])
ax.yaxis.set_ticklabels(['EOSINOPHIL', 'LYMPHOCYTE', 'MONOCYTE', 'NEUTROPHIL'])

In [ ]:
from sklearn.metrics import classification_report

print(classification_report(y_true, y_preds, target_names = ['EOSINOPHIL', 'LYMPHOCYTE', 'MONOCYTE', 'NEUTROPHIL']))

In [ ]:
model.save("./model.h5")